In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS
from multi_gpu import make_parallel

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


#### Building N-grams Neural Network

In [24]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation
from keras.layers import Flatten

## Decapitation of previous trained CNN
cnn = load_model('model_random_weights.keras')
input = cnn.layers[0].input
output = cnn.layers[-4].output
base_ngrams = Model(input=input, output=output)


ngrams = Sequential()
ngrams.add(base_ngrams)
ngrams.add(Dense(10000, activation='sigmoid', name= "10kdense")) 

ngrams.compile(optimizer='adam', loss='mse')

In [25]:
## Parellelize model 
p_model_ngrams = make_parallel(ngrams, 4)
p_model_ngrams.compile(optimizer='adam', loss='mse')

### Loading Images

In [11]:
from tqdm import tqdm, tqdm_notebook
base_dir = '/mnt/mjsynth/mnt/ramdisk/max/90kDICT32px/'
images_paths = get_image_paths(base_dir)
#%%time
ims = []
errors_1, errors_2 = [], []
for i, im in enumerate(images_paths[:12]):
    try:
        ims.append(preprocess_image(img_to_array(load_img(im, grayscale=True))))
    except OSError:
        errors_1.append(i)

In [12]:
batch = np.stack(ims).reshape(12, 32, 100, 1)
batch.shape

(12, 32, 100, 1)

In [13]:
from utils import NgramTransform

In [14]:
ngramtr = NgramTransform()

In [36]:
batch_y = ngramtr.make_batch_labels(images_paths[:12], False)
batch_y = batch_y.reshape(batch_y.shape[0], batch_y.shape[2])

In [54]:
ngrams.fit(batch, batch_y, nb_epoch=30)

Epoch 1/30
12/12 [==============================] - 1s - loss: 0.0026
Epoch 2/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 3/30
12/12 [==============================] - 0s - loss: 0.0027
Epoch 4/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 5/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 6/30
12/12 [==============================] - 1s - loss: 0.0027
Epoch 7/30
12/12 [==============================] - 0s - loss: 0.0027
Epoch 8/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 9/30
12/12 [==============================] - 0s - loss: 0.0027
Epoch 10/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 11/30
12/12 [==============================] - 1s - loss: 0.0027
Epoch 12/30
12/12 [==============================] - 0s - loss: 0.0027
Epoch 13/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 14/30
12/12 [==============================] - 0s - loss: 0.0026
Epoch 15/30
12/

In [55]:
predictions = ngrams.predict(batch)

In [56]:
for i in range(12):
#     print(ngramtr.ngram_from_matrix(batch_y[i]))
    print("{} \t\t {}".format(word_from_image_path(images_paths[:12][i]).lower(), ngramtr.ngram_from_matrix(predictions[i])) )

buckram 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
uncleanlier 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
choosing 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
soy 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
urinating 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
foreordaining 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
hmong 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
undercoat 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
rewedding 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
irrelevances 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
nasalized 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']
sup 		 ['a', 'd', 'e', 'i', 'in', 'ing', 'n', 'ng', 'o', 'r', 's']


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])